In [9]:
# Cell 1 — Setup, load data, helpers
# Install (once per session)
!pip install --quiet statsmodels scikit-learn numpy pandas tqdm

import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import Counter

from sklearn.metrics import f1_score, roc_auc_score
from statsmodels.stats.contingency_tables import mcnemar

# ---------- McNemar (paired) ----------
def mcnemar_test(y_true, y_pred_a, y_pred_b, exact_threshold=25):
    """
    Returns: dict with b, c, p_value, test_used
    b = A correct, B wrong
    c = A wrong,   B correct
    """
    y_true = np.asarray(y_true)
    a = (y_pred_a == y_true)
    b = (y_pred_b == y_true)

    b_count = int(np.sum(a & ~b))  # A correct, B wrong
    c_count = int(np.sum(~a & b))  # A wrong,   B correct

    exact = (b_count + c_count) < exact_threshold
    res = mcnemar([[0, b_count],[c_count, 0]], exact=exact, correction=not exact)
    return {
        "b": b_count, "c": c_count,
        "p_value": float(res.pvalue),
        "test_used": "exact" if exact else "chi2 (with continuity correction)"
    }

# ---------- Grouped bootstrap on ΔF1 (primary for CIU) ----------
def grouped_bootstrap_delta_f1(sample_ids, y_true, y_pred_a, y_pred_b, B=2000, random_state=42):
    """
    Resample by sample_id (with replacement), pool all tokens from selected samples,
    compute ΔF1 = F1(A) - F1(B) on each bootstrap. Returns CI and p-value.
    """
    rng = np.random.default_rng(random_state)
    sample_ids = np.asarray(sample_ids)
    y_true = np.asarray(y_true)
    y_pred_a = np.asarray(y_pred_a)
    y_pred_b = np.asarray(y_pred_b)

    # index tokens by sample_id
    sid_to_idx = {}
    for i, sid in enumerate(sample_ids):
        sid_to_idx.setdefault(sid, []).append(i)
    unique_sids = np.array(list(sid_to_idx.keys()))
    S = len(unique_sids)

    deltas = np.empty(B, dtype=float)
    for b in range(B):
        # sample S IDs with replacement
        draw = rng.choice(unique_sids, size=S, replace=True)
        idx = np.concatenate([sid_to_idx[sid] for sid in draw])
        f1_a = f1_score(y_true[idx], y_pred_a[idx])
        f1_b = f1_score(y_true[idx], y_pred_b[idx])
        deltas[b] = f1_a - f1_b

    # 95% CI
    lo, hi = np.percentile(deltas, [2.5, 97.5])
    # two-sided p ≈ 2 * min(Pr(Δ<=0), Pr(Δ>=0))
    p_left  = np.mean(deltas <= 0.0)
    p_right = np.mean(deltas >= 0.0)
    p_two   = 2 * min(p_left, p_right)
    return {
        "delta_mean": float(np.mean(deltas)),
        "delta_ci95": (float(lo), float(hi)),
        "p_value": float(min(1.0, p_two)),
        "dist": deltas  # keep if you want to plot later
    }

# ---------- DeLong test for ROC-AUC (paired) ----------
# Minimal implementation adapted for binary labels on the same examples.
# If your models don't output probabilities, pass decision_function scores (we'll still use DeLong).
def _compute_midrank(x):
    J = np.argsort(x)
    Z = x[J]
    N = len(x)
    T = np.zeros(N, dtype=float)
    i = 0
    while i < N:
        j = i
        while j < N and Z[j] == Z[i]:
            j += 1
        T[i:j] = 0.5*(i + j - 1) + 1
        i = j
    out = np.empty(N, dtype=float)
    out[J] = T
    return out

def _fast_delong(targets, predictions):
    # targets: 1/0 labels; predictions: scores
    # returns auc, auc_cov
    pos = predictions[targets == 1]
    neg = predictions[targets == 0]
    m = len(pos); n = len(neg)
    ranked = _compute_midrank(np.concatenate([pos, neg]))
    tx = ranked[:m]
    ty = ranked[m:]
    auc = (tx.sum() - m*(m+1)/2) / (m*n)

    v01 = (tx - (m+1)/2) / n
    v10 = 1 - (ty - (m+1)/2) / m
    sx = np.var(v01, ddof=1)
    sy = np.var(v10, ddof=1)
    auc_var = sx/m + sy/n
    return auc, auc_var

def delong_test(y_true, score_a, score_b):
    y_true = np.asarray(y_true).astype(int)
    score_a = np.asarray(score_a, dtype=float)
    score_b = np.asarray(score_b, dtype=float)
    auc_a, var_a = _fast_delong(y_true, score_a)
    auc_b, var_b = _fast_delong(y_true, score_b)
    # covariance approx (independent predictions → conservative)
    z = (auc_a - auc_b) / np.sqrt(var_a + var_b + 1e-12)
    # two-sided p-value under normal approx
    from math import erf, sqrt
    p = 2 * (1 - 0.5 * (1 + erf(abs(z)/sqrt(2))))
    return {
        "auc_a": float(auc_a),
        "auc_b": float(auc_b),
        "delta_auc": float(auc_a - auc_b),
        "z": float(z),
        "p_value": float(p)
    }

# ---------- Holm–Bonferroni ----------
def holm_bonferroni(pairs):
    """
    pairs: list of (label, pval). Returns DataFrame with adjusted decisions.
    """
    df = pd.DataFrame(pairs, columns=["label","p_raw"]).sort_values("p_raw").reset_index(drop=True)
    m = len(df)
    adj = []
    for i, p in enumerate(df["p_raw"], start=1):
        adj.append(min(1.0, (m - i + 1) * p))
    df["p_holm"] = np.maximum.accumulate(adj)  # monotone
    return df


In [12]:
# # Cell 2 — Ablation builder, train/eval loop, run grid, save metrics/models
# --- CONFIG paths (adjust as needed) ---
from google.colab import drive
drive.mount('/content/drive')

CSV_PATH = '/content/drive/MyDrive/aphasia/aphasia_tokens.csv'
SAVE_DIR = '/content/drive/MyDrive/aphasia/ablation_classic'

# --- Imports ---
!pip install --quiet pandas scikit-learn tqdm scipy joblib

import os, json, datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from joblib import dump

from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import (accuracy_score, precision_recall_fscore_support,
                             roc_auc_score, confusion_matrix)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from scipy import sparse

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

# --- Repro & output dir ---
SEED = 42
np.random.seed(SEED)
os.makedirs(SAVE_DIR, exist_ok=True)
STAMP = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
print("SAVE_DIR:", SAVE_DIR)

# --- Load CSV ---
df = pd.read_csv(CSV_PATH)
assert {'sample_id','token','is_word','is_CIU'}.issubset(df.columns), \
    "CSV must include: sample_id, token, is_word, is_CIU"

# --- Build context (±1, ±2) ---
def add_context(df):
    rows = []
    for sid, g in df.groupby('sample_id', sort=False):
        toks = g['token'].astype(str).tolist()
        isw  = g['is_word'].astype(int).tolist()
        iciu = g['is_CIU' ].astype(int).tolist()
        n = len(toks)
        for i in range(n):
            rows.append({
                'sample_id': sid,
                'token': toks[i],
                'prev1': toks[i-1] if i-1 >= 0 else '<BOS>',
                'next1': toks[i+1] if i+1 < n else '<EOS>',
                'prev2': toks[i-2] if i-2 >= 0 else '<BOS2>',
                'next2': toks[i+2] if i+2 < n else '<EOS2>',
                'is_word': isw[i],
                'is_CIU' : iciu[i],
            })
    return pd.DataFrame(rows)

X_all = add_context(df)

# --- Handcrafted features ---
STOPWORDS = {
    'the','a','an','and','but','or','so','to','of','in','on','at','for','from','with','by','as',
    'that','this','these','those','it','its','is','was','are','were','be','been','being','do','does','did',
    'have','has','had','he','she','they','we','you','i','me','him','her','them','us','my','your','our',
    'their','his','hers','theirs','mine','yours','ours','not','no','if','then','because','about','over',
    'under','up','down','out','into','off','just','very','really','there','here','now','also','too','again'
}
FILLERS = {'uh','um','er','ah','oh','mm','hmm','yeah','yep','nope','okay','ok','alright'}

class HandcraftedFeats(BaseEstimator, TransformerMixin):
    def __init__(self, use_context=True): self.use_context = use_context
    def fit(self, X, y=None): return self
    def _counts(self, s):
        al = sum(ch.isalpha() for ch in s)
        di = sum(ch.isdigit() for ch in s)
        pu = sum((not ch.isalnum()) for ch in s)
        return al, di, pu
    def transform(self, X):
        if isinstance(X, pd.DataFrame):
            token = X['token'].astype(str).values
            prev1 = X['prev1'].astype(str).values if 'prev1' in X else np.array(['']*len(X))
            next1 = X['next1'].astype(str).values if 'next1' in X else np.array(['']*len(X))
        else:
            token, prev1, next1 = X[:,0], X[:,1], X[:,2]
        rows = []
        for t, p, n in zip(token, prev1, next1):
            a,d,pu = self._counts(t)
            feats = [
                len(t), a, d, pu,
                float(t.isalpha()),
                float(t.islower()),
                float(t.isupper()),
                float("'" in t),
                float("-" in t or "–" in t),
                float(any(ch.isdigit() for ch in t)),
                float(t.lower() in STOPWORDS),
                float(t.lower() in FILLERS),
                float(t.lower() == 'and'),
                float(t.lower().startswith('&=')),
                float(t.lower() in {'xxx','xx'}),
                float(t.endswith('-')),
            ]
            if self.use_context:
                feats += [
                    float(p.lower() in STOPWORDS),
                    float(n.lower() in STOPWORDS),
                    float(p.lower() in FILLERS),
                    float(n.lower() in FILLERS),
                ]
            rows.append(feats)
        return sparse.csr_matrix(np.asarray(rows, dtype=np.float32))

# --- Model zoo ---
def model_zoo():
    return {
        'SVM-linear':  SVC(kernel='linear', probability=True, class_weight='balanced', random_state=SEED),
        'SVM-rbf':     SVC(kernel='rbf',   probability=True, class_weight='balanced', C=2.0, gamma='scale', random_state=SEED),
        'RandomForest': RandomForestClassifier(
            n_estimators=300, max_depth=None, class_weight='balanced_subsample', n_jobs=-1, random_state=SEED
        ),
        'DecisionTree': DecisionTreeClassifier(max_depth=None, class_weight='balanced', random_state=SEED),
        'KNN':          KNeighborsClassifier(n_neighbors=15, weights='distance')
    }

# --- Build ColumnTransformer from flags ---
def build_preprocessor(USE_TOKEN_CHAR=True, USE_CONTEXT_CHAR=True, USE_HANDCRAFTED=True, CTX_WINDOW=1):
    v_tok   = TfidfVectorizer(analyzer='char', ngram_range=(2,5), lowercase=True, min_df=1)
    v_prev1 = TfidfVectorizer(analyzer='char', ngram_range=(2,5), lowercase=True, min_df=1)
    v_next1 = TfidfVectorizer(analyzer='char', ngram_range=(2,5), lowercase=True, min_df=1)
    v_prev2 = TfidfVectorizer(analyzer='char', ngram_range=(2,5), lowercase=True, min_df=1)
    v_next2 = TfidfVectorizer(analyzer='char', ngram_range=(2,5), lowercase=True, min_df=1)

    transforms = []
    # token char n-grams
    if USE_TOKEN_CHAR:
        transforms.append(('tok_char', v_tok, 'token'))

    # context char n-grams
    if USE_CONTEXT_CHAR and CTX_WINDOW >= 1:
        transforms += [('prev_char', v_prev1, 'prev1'),
                       ('next_char', v_next1, 'next1')]
    if USE_CONTEXT_CHAR and CTX_WINDOW >= 2:
        transforms += [('prev2_char', v_prev2, 'prev2'),
                       ('next2_char', v_next2, 'next2')]

    # handcrafted features: only select columns that will actually be present
    if USE_HANDCRAFTED:
        hand_cols = ['token']
        if CTX_WINDOW >= 1:
            hand_cols += ['prev1','next1']
        transforms.append(('hand', HandcraftedFeats(use_context=(CTX_WINDOW >= 1)), hand_cols))

    return ColumnTransformer(transforms, remainder='drop', sparse_threshold=1.0)

# --- Fixed 80/20 split by sample_id (reuse across ablations) ---
X_df = X_all[['sample_id','token','prev1','next1','prev2','next2']].copy()
y_word = X_all['is_word'].astype(int).values
y_ciu  = X_all['is_CIU' ].astype(int).values
groups = X_all['sample_id'].values

gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=SEED)
(train_idx, test_idx), = gss.split(X_df, y_word, groups=groups)
X_train = X_df.iloc[train_idx].reset_index(drop=True)
X_test  = X_df.iloc[test_idx ].reset_index(drop=True)
y_word_tr, y_word_te = y_word[train_idx], y_word[test_idx]
y_ciu_tr,  y_ciu_te  = y_ciu [train_idx], y_ciu [test_idx]

print(f"Train tokens: {len(X_train)} | Test tokens: {len(X_test)}")
print(f"Train samples: {X_train['sample_id'].nunique()} | Test samples: {X_test['sample_id'].nunique()}")

# --- Evaluation helper (metrics only) ---
def evaluate(pipe, Xte, yte):
    yhat = pipe.predict(Xte)
    # For AUC:
    if hasattr(pipe.named_steps['clf'], 'predict_proba'):
        yscore = pipe.predict_proba(Xte)[:,1]
    elif hasattr(pipe.named_steps['clf'], 'decision_function'):
        s = pipe.decision_function(Xte)
        yscore = (s - s.min()) / (s.max() - s.min() + 1e-9)
    else:
        yscore = (yhat == 1).astype(float)
    acc = accuracy_score(yte, yhat)
    prec, rec, f1, _ = precision_recall_fscore_support(yte, yhat, average='binary', zero_division=0)
    try:
        auc = roc_auc_score(yte, yscore)
    except Exception:
        auc = float('nan')
    cm = confusion_matrix(yte, yhat).tolist()
    return dict(acc=acc, prec=prec, rec=rec, f1=f1, auc=auc, cm=cm)

# --- Run one ablation config across all models & tasks and SAVE each pipeline ---
def run_ablation(cfg):
    # only pass relevant keys to build_preprocessor
    cfg_bp = {k: cfg[k] for k in ['USE_TOKEN_CHAR','USE_CONTEXT_CHAR','USE_HANDCRAFTED','CTX_WINDOW'] if k in cfg}
    prep = build_preprocessor(**cfg_bp)

    # pick columns based on CTX_WINDOW
    cols = ['token']
    if cfg_bp.get('USE_CONTEXT_CHAR', True):
        if cfg_bp.get('CTX_WINDOW', 1) >= 1:
            cols += ['prev1','next1']
        if cfg_bp.get('CTX_WINDOW', 1) >= 2:
            cols += ['prev2','next2']

    rows = []
    for task, (y_tr, y_te) in {'WORD': (y_word_tr, y_word_te), 'CIU': (y_ciu_tr, y_ciu_te)}.items():
        for mdl_name, clf in model_zoo().items():
            pipe = Pipeline([('prep', prep), ('clf', clf)])
            pipe.fit(X_train[cols], y_tr)

            tag = cfg.get('name','cfg')
            model_path = os.path.join(SAVE_DIR, f"{STAMP}_{task}_{mdl_name}_{tag}.joblib")
            dump(pipe, model_path, compress=('xz', 3))

            # evaluate
            yhat = pipe.predict(X_test[cols])
            if hasattr(pipe.named_steps['clf'], 'predict_proba'):
                yscore = pipe.predict_proba(X_test[cols])[:,1]
            elif hasattr(pipe.named_steps['clf'], 'decision_function'):
                s = pipe.decision_function(X_test[cols])
                yscore = (s - s.min()) / (s.max() - s.min() + 1e-9)
            else:
                yscore = (yhat == 1).astype(float)

            acc  = accuracy_score(y_te, yhat)
            prec, rec, f1, _ = precision_recall_fscore_support(y_te, yhat, average='binary', zero_division=0)
            try:
                auc = roc_auc_score(y_te, yscore)
            except Exception:
                auc = float('nan')
            cm = confusion_matrix(y_te, yhat).tolist()

            row = {
                'timestamp': STAMP,
                'config': json.dumps(cfg, sort_keys=True),
                'config_name': tag,
                'task': task,
                'model': mdl_name,
                'acc': acc, 'prec': prec, 'rec': rec, 'f1': f1, 'auc': auc,
                'confusion': json.dumps(cm),
                'model_path': model_path,
                'train_tokens': len(X_train),
                'test_tokens':  len(X_test),
                'train_samples': X_train['sample_id'].nunique(),
                'test_samples':  X_test['sample_id'].nunique(),
            }
            print(f"[{task} | {mdl_name} | {tag}] F1={f1:.4f} Acc={acc:.4f} AUC={auc:.4f}  → {model_path}")
            rows.append(row)
    return rows

# --- Define ablation grid (includes 'baseline') ---
ablation_grid = [
    dict(USE_TOKEN_CHAR=True, USE_CONTEXT_CHAR=True,  USE_HANDCRAFTED=True,  CTX_WINDOW=1, name='baseline'),
    dict(USE_TOKEN_CHAR=False,USE_CONTEXT_CHAR=True,  USE_HANDCRAFTED=True,  CTX_WINDOW=1, name='-token_char'),
    dict(USE_TOKEN_CHAR=True, USE_CONTEXT_CHAR=False, USE_HANDCRAFTED=True,  CTX_WINDOW=0, name='-context_char'),
    dict(USE_TOKEN_CHAR=True, USE_CONTEXT_CHAR=True,  USE_HANDCRAFTED=False, CTX_WINDOW=1, name='-handcrafted'),
    dict(USE_TOKEN_CHAR=True, USE_CONTEXT_CHAR=True,  USE_HANDCRAFTED=True,  CTX_WINDOW=0, name='-context_window'),
    dict(USE_TOKEN_CHAR=True, USE_CONTEXT_CHAR=True,  USE_HANDCRAFTED=True,  CTX_WINDOW=2, name='+ctx2'),
]

# --- Train & save everything; write metrics CSV ---
all_rows = []
for cfg in ablation_grid:
    all_rows.extend(run_ablation(cfg))

metrics_df = pd.DataFrame(all_rows)
metrics_path = os.path.join(SAVE_DIR, f"{STAMP}_ablation_metrics.csv")
metrics_df.to_csv(metrics_path, index=False)
print("\nSaved metrics to:", metrics_path)

# quick listing
!ls -lah "$SAVE_DIR"



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
SAVE_DIR: /content/drive/MyDrive/aphasia/ablation_classic
Train tokens: 1755 | Test tokens: 222
Train samples: 28 | Test samples: 7
[WORD | SVM-linear | baseline] F1=0.9977 Acc=0.9955 AUC=0.9946  → /content/drive/MyDrive/aphasia/ablation_classic/2025-10-21_10-21-49_WORD_SVM-linear_baseline.joblib
[WORD | SVM-rbf | baseline] F1=0.9977 Acc=0.9955 AUC=0.9969  → /content/drive/MyDrive/aphasia/ablation_classic/2025-10-21_10-21-49_WORD_SVM-rbf_baseline.joblib
[WORD | RandomForest | baseline] F1=0.9977 Acc=0.9955 AUC=0.9892  → /content/drive/MyDrive/aphasia/ablation_classic/2025-10-21_10-21-49_WORD_RandomForest_baseline.joblib
[WORD | DecisionTree | baseline] F1=0.9977 Acc=0.9955 AUC=0.9167  → /content/drive/MyDrive/aphasia/ablation_classic/2025-10-21_10-21-49_WORD_DecisionTree_baseline.joblib
[WORD | KNN | baseline] F1=0.9977 Acc=0.9955 AUC=0.9136  → /content/drive

In [13]:
# Cell A — Stats utilities
# ========== Stats utilities: McNemar, grouped bootstrap ΔF1, DeLong, Holm–Bonferroni ==========

!pip install --quiet statsmodels

import numpy as np
import pandas as pd
from statsmodels.stats.contingency_tables import mcnemar

# McNemar on accuracy (paired 2x2)
def mcnemar_test(y_true, y_pred_a, y_pred_b, exact_threshold=25):
    y_true = np.asarray(y_true)
    a = (np.asarray(y_pred_a) == y_true)
    b = (np.asarray(y_pred_b) == y_true)
    b_count = int(np.sum(a & ~b))  # A correct, B wrong
    c_count = int(np.sum(~a & b))  # A wrong,   B correct
    exact = (b_count + c_count) < exact_threshold
    res = mcnemar([[0, b_count], [c_count, 0]], exact=exact, correction=not exact)
    return {
        "b": b_count, "c": c_count,
        "p_value": float(res.pvalue),
        "test_used": "exact" if exact else "chi2_corr"
    }

# Grouped bootstrap ΔF1 by sample_id (primary for CIU)
from sklearn.metrics import f1_score
def grouped_bootstrap_delta_f1(sample_ids, y_true, y_pred_a, y_pred_b, B=2000, random_state=42):
    rng = np.random.default_rng(random_state)
    sample_ids = np.asarray(sample_ids)
    y_true     = np.asarray(y_true)
    y_pred_a   = np.asarray(y_pred_a)
    y_pred_b   = np.asarray(y_pred_b)

    sid_to_idx = {}
    for i, sid in enumerate(sample_ids):
        sid_to_idx.setdefault(sid, []).append(i)
    unique_sids = np.array(list(sid_to_idx.keys()))
    S = len(unique_sids)

    deltas = np.empty(B, dtype=float)
    for b in range(B):
        draw = rng.choice(unique_sids, size=S, replace=True)
        idx = np.concatenate([sid_to_idx[sid] for sid in draw])
        f1_a = f1_score(y_true[idx], y_pred_a[idx])
        f1_b = f1_score(y_true[idx], y_pred_b[idx])
        deltas[b] = f1_a - f1_b

    lo, hi = np.percentile(deltas, [2.5, 97.5])
    p_left  = np.mean(deltas <= 0.0)
    p_right = np.mean(deltas >= 0.0)
    p_two   = min(1.0, 2 * min(p_left, p_right))
    return {
        "delta_mean": float(np.mean(deltas)),
        "ci_lo": float(lo),
        "ci_hi": float(hi),
        "p_value": float(p_two),
    }

# Minimal paired DeLong AUC comparison (binary)
from sklearn.metrics import roc_auc_score
def _compute_midrank(x):
    J = np.argsort(x)
    Z = x[J]
    N = len(x)
    T = np.zeros(N, dtype=float)
    i = 0
    while i < N:
        j = i
        while j < N and Z[j] == Z[i]:
            j += 1
        T[i:j] = 0.5*(i + j - 1) + 1
        i = j
    out = np.empty(N, dtype=float)
    out[J] = T
    return out

def _fast_delong(targets, predictions):
    pos = predictions[targets == 1]
    neg = predictions[targets == 0]
    m = len(pos); n = len(neg)
    ranked = _compute_midrank(np.concatenate([pos, neg]))
    tx = ranked[:m]; ty = ranked[m:]
    auc = (tx.sum() - m*(m+1)/2) / (m*n)
    v01 = (tx - (m+1)/2) / n
    v10 = 1 - (ty - (m+1)/2) / m
    sx = np.var(v01, ddof=1)
    sy = np.var(v10, ddof=1)
    auc_var = sx/m + sy/n
    return auc, auc_var

def delong_test(y_true, score_a, score_b):
    y_true  = np.asarray(y_true).astype(int)
    score_a = np.asarray(score_a, dtype=float)
    score_b = np.asarray(score_b, dtype=float)
    auc_a, var_a = _fast_delong(y_true, score_a)
    auc_b, var_b = _fast_delong(y_true, score_b)
    z = (auc_a - auc_b) / np.sqrt(var_a + var_b + 1e-12)
    from math import erf, sqrt
    p = 2 * (1 - 0.5 * (1 + erf(abs(z)/sqrt(2))))
    return {"auc_a": float(auc_a), "auc_b": float(auc_b),
            "delta_auc": float(auc_a - auc_b), "z": float(z), "p_value": float(p)}

# Holm–Bonferroni correction over multiple ablations (primary metric p-values)
def holm_bonferroni(pairs):
    df = pd.DataFrame(pairs, columns=["label","p_raw"]).sort_values("p_raw").reset_index(drop=True)
    m = len(df)
    adj = []
    for i, p in enumerate(df["p_raw"], start=1):
        adj.append(min(1.0, (m - i + 1) * p))
    # monotone non-decreasing adjusted p-values
    df["p_holm"] = np.maximum.accumulate(adj)
    return df


In [18]:
# Cell B — Integrate stats into the ablation runner and save tables
# ========== Extend ablation runner with stats and table outputs ==========

from joblib import dump, load
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import (accuracy_score, precision_recall_fscore_support,
                             roc_auc_score, confusion_matrix)

# --- Reuse: X_all exists from your previous cells ---
# Build fixed 80/20 split once
X_df = X_all[['sample_id','token','prev1','next1','prev2','next2']].copy()
y_word = X_all['is_word'].astype(int).values
y_ciu  = X_all['is_CIU' ].astype(int).values
groups = X_all['sample_id'].values

gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=SEED)
(train_idx, test_idx), = gss.split(X_df, y_word, groups=groups)
X_train = X_df.iloc[train_idx].reset_index(drop=True)
X_test  = X_df.iloc[test_idx ].reset_index(drop=True)
y_word_tr, y_word_te = y_word[train_idx], y_word[test_idx]
y_ciu_tr,  y_ciu_te  = y_ciu [train_idx], y_ciu [test_idx]
groups_te = X_test['sample_id'].values

print(f"[Split] Train tokens: {len(X_train)} | Test tokens: {len(X_test)} | "
      f"Train samples: {X_train['sample_id'].nunique()} | Test samples: {X_test['sample_id'].nunique()}")

# --- Build preprocessor remains as in Cell 2 previously ---
def build_preprocessor(USE_TOKEN_CHAR=True, USE_CONTEXT_CHAR=True, USE_HANDCRAFTED=True, CTX_WINDOW=1):
    v_tok   = TfidfVectorizer(analyzer='char', ngram_range=(2,5), lowercase=True, min_df=1)
    v_prev1 = TfidfVectorizer(analyzer='char', ngram_range=(2,5), lowercase=True, min_df=1)
    v_next1 = TfidfVectorizer(analyzer='char', ngram_range=(2,5), lowercase=True, min_df=1)
    v_prev2 = TfidfVectorizer(analyzer='char', ngram_range=(2,5), lowercase=True, min_df=1)
    v_next2 = TfidfVectorizer(analyzer='char', ngram_range=(2,5), lowercase=True, min_df=1)

    transforms = []
    if USE_TOKEN_CHAR:
        transforms.append(('tok_char', v_tok, 'token'))

    if USE_CONTEXT_CHAR and CTX_WINDOW >= 1:
        transforms += [('prev_char', v_prev1, 'prev1'),
                       ('next_char', v_next1, 'next1')]
    if USE_CONTEXT_CHAR and CTX_WINDOW >= 2:
        transforms += [('prev2_char', v_prev2, 'prev2'),
                       ('next2_char', v_next2, 'next2')]

    if USE_HANDCRAFTED:
        hand_cols = ['token']
        if CTX_WINDOW >= 1:
            hand_cols += ['prev1','next1']
        transforms.append(('hand', HandcraftedFeats(use_context=(CTX_WINDOW >= 1)), hand_cols))

    return ColumnTransformer(transforms, remainder='drop', sparse_threshold=1.0)

# --- Simple model zoo (as before) ---
def model_zoo():
    return {
        'SVM-linear':  SVC(kernel='linear', probability=True, class_weight='balanced', random_state=SEED),
        'SVM-rbf':     SVC(kernel='rbf',   probability=True, class_weight='balanced', C=2.0, gamma='scale', random_state=SEED),
        'RandomForest': RandomForestClassifier(
            n_estimators=300, max_depth=None, class_weight='balanced_subsample', n_jobs=-1, random_state=SEED
        ),
        'DecisionTree': DecisionTreeClassifier(max_depth=None, class_weight='balanced', random_state=SEED),
        'KNN':          KNeighborsClassifier(n_neighbors=15, weights='distance')
    }

# --- Evaluate returns metrics + predictions + scores (for stats) ---
def evaluate_with_preds(pipe, Xte, yte):
    yhat = pipe.predict(Xte)
    # scores for AUC/DeLong
    if hasattr(pipe.named_steps['clf'], "predict_proba"):
        yscore = pipe.predict_proba(Xte)[:,1]
    elif hasattr(pipe.named_steps['clf'], "decision_function"):
        s = pipe.decision_function(Xte)
        yscore = (s - s.min()) / (s.max() - s.min() + 1e-9)  # monotone scaling
    else:
        yscore = (yhat == 1).astype(float)
    acc = accuracy_score(yte, yhat)
    prec, rec, f1, _ = precision_recall_fscore_support(yte, yhat, average='binary', zero_division=0)
    try:
        auc = roc_auc_score(yte, yscore)
    except Exception:
        auc = float('nan')
    cm = confusion_matrix(yte, yhat)
    return dict(acc=acc, prec=prec, rec=rec, f1=f1, auc=auc,
                cm=cm.tolist(), yhat=yhat, yscore=yscore)

# --- Run ablation: returns rows (for metrics CSV) and stats_data (for later comparisons) ---
def run_ablation(cfg):
    # Only pass relevant keys to build_preprocessor
    cfg_bp = {k: cfg[k] for k in ['USE_TOKEN_CHAR','USE_CONTEXT_CHAR','USE_HANDCRAFTED','CTX_WINDOW'] if k in cfg}
    prep = build_preprocessor(**cfg_bp)

    # Choose columns to feed based on CTX_WINDOW
    cols = ['token']
    if cfg_bp.get('USE_CONTEXT_CHAR', True):
        if cfg_bp.get('CTX_WINDOW', 1) >= 1:
            cols += ['prev1','next1']
        if cfg_bp.get('CTX_WINDOW', 1) >= 2:
            cols += ['prev2','next2']

    results_rows = []
    stats_data   = []

    for task_name, (y_tr, y_te) in {'WORD': (y_word_tr, y_word_te), 'CIU': (y_ciu_tr, y_ciu_te)}.items():
        for mdl_name, clf in model_zoo().items():
            pipe = Pipeline([('prep', prep), ('clf', clf)])
            pipe.fit(X_train[cols], y_tr)

            tag = cfg.get('name','cfg')
            model_path = os.path.join(SAVE_DIR, f"{STAMP}_{task_name}_{mdl_name}_{tag}.joblib")
            dump(pipe, model_path, compress=('xz', 3))

            # Evaluate with preds/scores
            yhat = pipe.predict(X_test[cols])
            if hasattr(pipe.named_steps['clf'], "predict_proba"):
                yscore = pipe.predict_proba(X_test[cols])[:,1]
            elif hasattr(pipe.named_steps['clf'], "decision_function"):
                s = pipe.decision_function(X_test[cols])
                yscore = (s - s.min()) / (s.max() - s.min() + 1e-9)
            else:
                yscore = (yhat == 1).astype(float)

            acc  = accuracy_score(y_te, yhat)
            prec, rec, f1, _ = precision_recall_fscore_support(y_te, yhat, average='binary', zero_division=0)
            try:
                auc = roc_auc_score(y_te, yscore)
            except Exception:
                auc = float('nan')
            cm = confusion_matrix(y_te, yhat).tolist()

            results_rows.append({
                'timestamp': STAMP,
                'config': json.dumps(cfg, sort_keys=True),
                'config_name': tag,
                'task': task_name,
                'model': mdl_name,
                'acc': acc, 'prec': prec, 'rec': rec, 'f1': f1, 'auc': auc,
                'confusion': json.dumps(cm),
                'model_path': model_path,
                'train_tokens': len(X_train),
                'test_tokens':  len(X_test),
                'train_samples': X_train['sample_id'].nunique(),
                'test_samples':  X_test['sample_id'].nunique(),
            })

            stats_data.append({
                'config_name': tag,
                'task': task_name,
                'model': mdl_name,
                'y_true': y_te,
                'y_hat':  yhat,
                'y_score': yscore,
                'groups': groups_te,  # sample_id per token
            })

            print(f"[{task_name} | {mdl_name} | {tag}] F1={f1:.4f} Acc={acc:.4f} AUC={auc:.4f}  → {model_path}")

    return results_rows, stats_data

# --- Define ablation grid (baseline + toggles) ---
ablation_grid = [
    dict(USE_TOKEN_CHAR=True, USE_CONTEXT_CHAR=True,  USE_HANDCRAFTED=True,  CTX_WINDOW=1, name='baseline'),
    dict(USE_TOKEN_CHAR=False,USE_CONTEXT_CHAR=True,  USE_HANDCRAFTED=True,  CTX_WINDOW=1, name='-token_char'),
    dict(USE_TOKEN_CHAR=True, USE_CONTEXT_CHAR=False, USE_HANDCRAFTED=True,  CTX_WINDOW=0, name='-context_char'),
    dict(USE_TOKEN_CHAR=True, USE_CONTEXT_CHAR=True,  USE_HANDCRAFTED=False, CTX_WINDOW=1, name='-handcrafted'),
    dict(USE_TOKEN_CHAR=True, USE_CONTEXT_CHAR=True,  USE_HANDCRAFTED=True,  CTX_WINDOW=0, name='-context_window'),
    dict(USE_TOKEN_CHAR=True, USE_CONTEXT_CHAR=True,  USE_HANDCRAFTED=True,  CTX_WINDOW=2, name='+ctx2'),
]

# Reset accumulators before rerun
all_rows = []
all_stats = []

for cfg in ablation_grid:
    rows, stats = run_ablation(cfg)
    all_rows.extend(rows)
    all_stats.extend(stats)

# Save per-run metrics
metrics_df = pd.DataFrame(all_rows)
metrics_path = os.path.join(SAVE_DIR, f"{STAMP}_ablation_metrics.csv")
metrics_df.to_csv(metrics_path, index=False)
print("Saved metrics to:", metrics_path)

# Build stats vs baseline...
stats_rows = []
for (task, model), group in pd.DataFrame(all_stats).groupby(['task','model']):
    base = group[group['config_name'] == 'baseline']
    if base.empty:
        continue
    base_row = base.iloc[0]
    y_true  = base_row['y_true']
    y_hat_A = base_row['y_hat']
    y_scr_A = base_row['y_score']
    sids    = base_row['groups']

    for _, r in group.iterrows():
        if r['config_name'] == 'baseline':
            continue
        y_hat_B = r['y_hat']
        y_scr_B = r['y_score']

        mc = mcnemar_test(y_true, y_hat_A, y_hat_B)
        boot = grouped_bootstrap_delta_f1(sids, y_true, y_hat_A, y_hat_B, B=2000, random_state=SEED)
        try:
            dl = delong_test(y_true, y_scr_A, y_scr_B)
        except Exception:
            dl = {"auc_a": np.nan, "auc_b": np.nan, "delta_auc": np.nan, "z": np.nan, "p_value": np.nan}

        stats_rows.append({
            'timestamp': STAMP,
            'task': task,
            'model': model,
            'compare_to': 'baseline',
            'ablation': r['config_name'],
            'F1_base': float(metrics_df[(metrics_df.task==task)&(metrics_df.model==model)&(metrics_df.config_name=='baseline')]['f1'].iloc[0]),
            'F1_ablt': float(metrics_df[(metrics_df.task==task)&(metrics_df.model==model)&(metrics_df.config_name==r['config_name'])]['f1'].iloc[0]),
            'ΔF1_mean_boot': boot['delta_mean'],
            'ΔF1_ci95_lo': boot['ci_lo'],
            'ΔF1_ci95_hi': boot['ci_hi'],
            'p_boot_ΔF1': boot['p_value'],
            'McNemar_b': mc['b'],
            'McNemar_c': mc['c'],
            'p_McNemar': mc['p_value'],
            'DeLong_AUC_base': dl['auc_a'],
            'DeLong_AUC_ablt': dl['auc_b'],
            'ΔAUC': dl['delta_auc'],
            'z_DeLong': dl['z'],
            'p_DeLong': dl['p_value'],
        })

stats_df = pd.DataFrame(stats_rows)
stats_path = os.path.join(SAVE_DIR, f"{STAMP}_ablation_stats_vs_baseline.csv")
stats_df.to_csv(stats_path, index=False)
print("Saved stats to:", stats_path)

# Holm–Bonferroni on CIU ΔF1 p-values per model
adj_rows = []
for model, g in stats_df[stats_df.task=="CIU"].groupby('model'):
    pairs = [(row['ablation'], row['p_boot_ΔF1']) for _, row in g.iterrows()]
    if not pairs:
        continue
    adj = holm_bonferroni(pairs)
    adj['task'] = 'CIU'
    adj['model'] = model
    adj_rows.append(adj)

holm_df = pd.concat(adj_rows, ignore_index=True) if adj_rows else pd.DataFrame(columns=['label','p_raw','p_holm','task','model'])
holm_path = os.path.join(SAVE_DIR, f"{STAMP}_holm_adjustment_CIU.csv")
holm_df.to_csv(holm_path, index=False)
print("Saved Holm–Bonferroni table (CIU ΔF1):", holm_path)

# Convenience views
print("\n--- Top ΔF1 (CIU) improvements (mean bootstrap) ---")
display(stats_df[stats_df.task=='CIU'].sort_values('ΔF1_mean_boot', ascending=False).head(10))

print("\n--- Holm–Bonferroni adjusted p-values (CIU ΔF1) ---")
display(holm_df.sort_values(['model','p_holm']))

[Split] Train tokens: 1755 | Test tokens: 222 | Train samples: 28 | Test samples: 7
[WORD | SVM-linear | baseline] F1=0.9977 Acc=0.9955 AUC=0.9946  → /content/drive/MyDrive/aphasia/ablation_classic/2025-10-21_10-21-49_WORD_SVM-linear_baseline.joblib
[WORD | SVM-rbf | baseline] F1=0.9977 Acc=0.9955 AUC=0.9969  → /content/drive/MyDrive/aphasia/ablation_classic/2025-10-21_10-21-49_WORD_SVM-rbf_baseline.joblib
[WORD | RandomForest | baseline] F1=0.9977 Acc=0.9955 AUC=0.9892  → /content/drive/MyDrive/aphasia/ablation_classic/2025-10-21_10-21-49_WORD_RandomForest_baseline.joblib
[WORD | DecisionTree | baseline] F1=0.9977 Acc=0.9955 AUC=0.9167  → /content/drive/MyDrive/aphasia/ablation_classic/2025-10-21_10-21-49_WORD_DecisionTree_baseline.joblib
[WORD | KNN | baseline] F1=0.9977 Acc=0.9955 AUC=0.9136  → /content/drive/MyDrive/aphasia/ablation_classic/2025-10-21_10-21-49_WORD_KNN_baseline.joblib
[CIU | SVM-linear | baseline] F1=0.8653 Acc=0.7883 AUC=0.7463  → /content/drive/MyDrive/aphasia/ab

,timestamp,task,model,compare_to,ablation,F1_base,F1_ablt,ΔF1_mean_boot,ΔF1_ci95_lo,ΔF1_ci95_hi,p_boot_ΔF1,McNemar_b,McNemar_c,p_McNemar,DeLong_AUC_base,DeLong_AUC_ablt,ΔAUC,z_DeLong,p_DeLong
14,2025-10-21_10-21-49,CIU,RandomForest,baseline,+ctx2,0.867052,0.819277,0.054183,0.027608,0.129633,0.000,19,5,0.006611,0.766018,0.731064,0.034954,0.226481,0.820827
6,2025-10-21_10-21-49,CIU,KNN,baseline,-context_char,0.888889,0.864407,0.040768,-0.030691,0.167429,0.424,32,23,0.280713,0.786991,0.765836,0.021155,0.139962,0.888690
8,2025-10-21_10-21-49,CIU,KNN,baseline,-context_window,0.888889,0.864407,0.040768,-0.030691,0.167429,0.424,32,23,0.280713,0.786991,0.765836,0.021155,0.139962,0.888690
5,2025-10-21_10-21-49,CIU,KNN,baseline,-token_char,0.888889,0.851312,0.035848,0.001194,0.064382,0.041,16,4,0.011818,0.786991,0.757021,0.029970,0.197265,0.843621
0,2025-10-21_10-21-49,CIU,DecisionTree,baseline,-token_char,0.796353,0.767296,0.029059,-0.009220,0.064167,0.164,31,24,0.418492,0.628207,0.632401,-0.004195,-0.029532,0.976440
12,2025-10-21_10-21-49,CIU,RandomForest,baseline,-handcrafted,0.867052,0.845238,0.027133,0.007483,0.076717,0.000,10,4,0.179565,0.766018,0.755076,0.010942,0.071382,0.943093
21,2025-10-21_10-21-49,CIU,SVM-rbf,baseline,-context_char,0.877493,0.854651,0.024094,0.003593,0.049647,0.010,12,5,0.143463,0.797264,0.749119,0.048146,0.317209,0.751085
23,2025-10-21_10-21-49,CIU,SVM-rbf,baseline,-context_window,0.877493,0.854651,0.024094,0.003593,0.049647,0.010,12,5,0.143463,0.797264,0.749119,0.048146,0.317209,0.751085
7,2025-10-21_10-21-49,CIU,KNN,baseline,-handcrafted,0.888889,0.874251,0.017937,-0.009944,0.048626,0.247,16,13,0.710347,0.786991,0.779271,0.007720,0.051162,0.959196
10,2025-10-21_10-21-49,CIU,RandomForest,baseline,-token_char,0.867052,0.848665,0.017198,-0.016990,0.045202,0.202,17,12,0.457614,0.766018,0.753921,0.012097,0.078896,0.937116



--- Holm–Bonferroni adjusted p-values (CIU ΔF1) ---


,label,p_raw,p_holm,task,model
0,-handcrafted,0.000,0.000,CIU,DecisionTree
1,-context_char,0.071,0.284,CIU,DecisionTree
2,-context_window,0.071,0.284,CIU,DecisionTree
3,-token_char,0.164,0.328,CIU,DecisionTree
4,+ctx2,0.763,0.763,CIU,DecisionTree
5,-token_char,0.041,0.205,CIU,KNN
6,-handcrafted,0.247,0.988,CIU,KNN
7,-context_char,0.424,1.000,CIU,KNN
8,-context_window,0.424,1.000,CIU,KNN
9,+ctx2,0.652,1.000,CIU,KNN


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
